In [4]:
from pprint import pprint
import pandas as pd
import sys, os
import random
import copy
import time


In [5]:
HUMAN = 1
COMP = -1

In [6]:
def init():
  grid = int(input("Chosir votre grille : 3(3x3), 5(5x5) ou 7(7x7)"))
  game = [[0 for j in range(grid)] for i in range(grid)]
  return game, grid


In [7]:
game, grid = init()

Chosir votre grille : 3(3x3), 5(5x5) ou 7(7x7)5


In [8]:
def format_game(df):
  df = df.replace([1],'X')
  df = df.replace([-1],'O')
  df = df.replace([0],'.')
  
  return df

In [9]:
df = pd.DataFrame(game)
print(format_game(df))

   0  1  2  3  4
0  .  .  .  .  .
1  .  .  .  .  .
2  .  .  .  .  .
3  .  .  .  .  .
4  .  .  .  .  .


In [10]:
def is_empty_box(game, x, y):
  if game[x][y] == 1 or game[x][y] == -1:
    return False
  return True

In [11]:
def is_fullfilled(game):
  for item in game:
    for elt in item:
      if elt == 0:
        return False
  return True                

In [12]:
def check_diago_top(state, posx, posy):
    """
    """
    x = posx
    y = posy
    
        
    try:
        if x - (comboforWin-1) >= 0 and y + (comboforWin-1) and comboforWin==3:
            diagoT =  [state[x][y], state[x-1][y+1], state[x-2][y+2]]
        elif x - (comboforWin-1) >= 0 and y + (comboforWin-1) < grid and comboforWin==4:
            diagoT =  [state[x][y], state[x-1][y+1], state[x-2][y+2],state[x-3][y+3] ]
        return diagoT
    except:
        pass
    
    return 

In [13]:
def check_the_column(state, posx, posy):
    """
    """
    column = []
    if posx + (comboforWin-1) < grid:
        for i in range(comboforWin):
            column.append(state[posx][posy])
            posx+=1
        return column
    return

In [14]:
def check_diago_bottom(state, posx, posy):
    """
    """
    x = posx
    y = posy
    try:
        if comboforWin == 3:
            diagoB =  [state[x][y], state[x+1][y+1], state[x+2][y+2]]
        elif comboforWin == 4:
            diagoB =  [state[x][y], state[x+1][y+1], state[x+2][y+2], state[x+3][y+3]]
        return diagoB
    except:
        pass
    
    return 

In [15]:
def check_the_line(state, posx, posy):
    """
    """
    line = []
    if posy + (comboforWin-1) < grid:
        for i in range(comboforWin):
            line.append(state[posx][posy+i])
        return line
    return

In [16]:
def genere_params_for_win(grilles):
    if grilles == 3:
        comboforWin = 3
    else:
        comboforWin = 4
    return comboforWin

In [17]:
comboforWin = genere_params_for_win(grid)

In [18]:
def create_win_possibility(state):
    wins_state = []
    for x in range(len(state)):
        for y in range(len(state)):
            line = check_the_line(state, x, y)
            if line:
                wins_state.append(line)
    
            column = check_the_column(state, x, y)
            if column:
                wins_state.append(column)
            diago_bottom = check_diago_bottom(state, x, y)
            if diago_bottom:
                wins_state.append(diago_bottom)
                
            diago_top = check_diago_top(state, x, y)
            if diago_top:
                wins_state.append(diago_top)
    return wins_state

In [19]:
def wins(state, player):
  
    win_state = create_win_possibility(state)
    if genere_params_for_win(grid) == 3 and [player, player, player] in win_state:
        return True
    elif genere_params_for_win(grid) == 4 and [player, player, player, player] in win_state:
        return True
    else:
        return False

In [20]:
def random_row_col():
  x = random.randint(0, grid-1)
  y = random.randint(0, grid-1)

  return x,y


In [21]:
def play_with_random(game):
  is_win = False
  player = 1
  count = 0
  while is_win == False:
    if count == 0:
      print(pd.DataFrame(game))

    if player == 1:
      print('A mon tour')
      x = int(input("Chosir l'abscisse"))
      y = int(input("Chosir l'ordonnée"))

    if player == -1:
      print("Au tour de l'IA")
      x = random.randint(0, grid-1)
      y = random.randint(0, grid-1)

    if is_empty_box(game, x, y) == False:
      while is_empty_box(game, x, y) == False:
        if player == 1:
          print("Case prise, veuillez réessayer")
          x = int(input("Chosir l'abscisse"))
          y = int(input("Chosir l'ordonnée"))
        if player == -1:
          x = random.randint(0, grid-1)
          y = random.randint(0, grid-1)

    game[x][y] = player
    show_ground = copy.deepcopy(game)
    count = count + 1
    
    is_win = wins(game, player)
    if count > 0:
      df = pd.DataFrame(show_ground)
      df = format_game(df)
      print(df)

    if is_win == True:
      name = ""
      if player == 1:
        name = "Humain"
      else:
        name = "Machine"
      print("{} a gagné".format(name))
    else:
      if is_fullfilled(game) == True:
        print("Match null")

    player = -player




In [22]:
#play_with_random(game)

In [23]:
def evaluate(state):
    """
    Perform heuristic evaluation from board.
    Heuristic - allow the computer to discover the solution
    of some problems by itself.
    """
    if wins(state, COMP):
        score = -1
    elif wins(state, HUMAN):
        score = 1
    else:
        score = 0

    return score

In [24]:
def empty_cells(state):
    """
    Identifier à chaque étape du jeu les cellules qui reste à cocher
    """
    cells = [] # it contains all empty cells

    # Use enumerate for easy indexing
    for i, row in enumerate(state):
        for j, col in enumerate(row):
            if state[i][j] == 0:
                cells.append([i, j])

    return cells

In [25]:
def game_over(state):
    """Check game over condition"""
    return wins(state, HUMAN) or wins(state, COMP)

In [26]:
evaluate(game)

0

In [27]:
def minimax(state, depth, player):

    from math import inf  
    if player == COMP:
        best = [-1, -1, inf] # inf/-inf are the initial score for the players
    else:
        best = [-1, -1, -inf]

    if depth == 0 or game_over(state):
        score = evaluate(state)
        return [-1, -1, score]
    
    # Limit depth for board 5x5 & 7x7
    if depth > 10:
        depth = 2

    for cell in empty_cells(state):
        # Fill the empty cells with the player symbols
        x, y = cell[0], cell[1]
        state[x][y] = player
        #print('\ncell : ', cell)
        #print('\nsymbole : ', player)
        score = minimax(state, depth-1, -player)
        #print('\nscore : ', score)
        state[x][y] = 0
        score[0], score[1] = x, y

        if player == COMP:
            if score[2] < best[2]:
                best = score
                #print('\nbest : ', best)
        else:
            if score[2] > best[2]:
                best = score
                #print('\nbest : ', best)

    return best


In [28]:
def ai_turn(state):
    depth = len(empty_cells(state)) # The remaining of empty cells
    row, col, score = minimax(state, depth, COMP) # the optimal move for computer
    #state[row][col] = COMP

    return row, col
    #print("A.I Turn")
    #print(render(state))

In [29]:
def main(game):
  is_end = False
  player = 1
  count = 0
  while is_end == False:
    if count == 0:
      print(format_game(pd.DataFrame(game)))

    if player == 1:
      print('A mon tour')
      x = int(input("Chosir ligne"))
      y = int(input("Chosir colonne"))

    if player == -1:
      x, y = ai_turn(game)

    if is_empty_box(game, x, y) == False:
      while is_empty_box(game, x, y) == False:
        if player == 1:
          print("Case prise, veuillez réessayer")
          x = int(input("Chosir ligne"))
          y = int(input("Chosir colonne"))
        if player == -1:
          print("Au tour de l'IA")
          x,y = ai_turn(game)
          
    game[x][y] = player
    show_ground = copy.deepcopy(game)
    count = count + 1
    
    is_win = wins(game, player)
    if count > 0:
      df = pd.DataFrame(show_ground)
      df = format_game(df)
      print(df)

    if is_win == True:
      is_end = True
      name = ""
      if player == 1:
        name = "Humain"
      else:
        name = "Machine"
      print("{} a gagné".format(name))
    else:
      if len(empty_cells(game)) == 0:
        print("Match null")
        is_end = True

    player = -player

In [30]:
def match(game, type_match):
  # type_match
  # 1 human vs random
  # 2 human vs minimax
  # 3 random vs minmax
  is_end = False
  player = 1
  count = 0
  while is_end == False:
    if count == 0:
      print(format_game(pd.DataFrame(game)))

    if player == 1:
      if type_match == 1 or type_match == 2:
        print('A mon tour')
        x = int(input("Chosir ligne"))
        y = int(input("Chosir colonne"))
      else:
        print('Au tour du random')
        x, y = random_row_col()  

    if player == -1:
        print("Au tour de l'IA") 
        x, y = ai_turn(game)

    if is_empty_box(game, x, y) == False:
      while is_empty_box(game, x, y) == False:
        if player == 1:
          print("Case prise, veuillez réessayer")
          x, y = random_row_col()
        if player == -1:
          print("Au tour de l'IA minmax")
          x,y = ai_turn(game)
          
    game[x][y] = player
    show_ground = copy.deepcopy(game)
    count = count + 1
    
    is_win = wins(game, player)
    if count > 0:
      df = pd.DataFrame(show_ground)
      df = format_game(df)
      print(df)

    if is_win == True:
      is_end = True
      name = ""
      if player == 1:
        if type_match == 1 or type_match == 2:
          name = "Humain"
        else:
          name: "Random"
      else:
          name = "IA"
      print("{} a gagné".format(name))
    else:
      if len(empty_cells(game)) == 0:
        print("Match null")
        is_end = True

    player = -player
    time.sleep(3)

In [31]:
if __name__ == '__main__':
    #main(game)
    #random_vs_minimax(game)
    # type_match
    # 1 human vs random
    # 2 human vs minimax
    # 3 random vs minmax
    match(game, 3)

   0  1  2  3  4
0  .  .  .  .  .
1  .  .  .  .  .
2  .  .  .  .  .
3  .  .  .  .  .
4  .  .  .  .  .
Au tour du random
   0  1  2  3  4
0  .  .  .  .  .
1  .  .  .  .  .
2  .  .  .  .  .
3  .  X  .  .  .
4  .  .  .  .  .
Au tour de l'IA
   0  1  2  3  4
0  O  .  .  .  .
1  .  .  .  .  .
2  .  .  .  .  .
3  .  X  .  .  .
4  .  .  .  .  .
Au tour du random
   0  1  2  3  4
0  O  .  .  .  .
1  .  .  .  .  .
2  .  .  .  .  .
3  .  X  .  .  X
4  .  .  .  .  .
Au tour de l'IA
   0  1  2  3  4
0  O  O  .  .  .
1  .  .  .  .  .
2  .  .  .  .  .
3  .  X  .  .  X
4  .  .  .  .  .
Au tour du random
   0  1  2  3  4
0  O  O  X  .  .
1  .  .  .  .  .
2  .  .  .  .  .
3  .  X  .  .  X
4  .  .  .  .  .
Au tour de l'IA
   0  1  2  3  4
0  O  O  X  O  .
1  .  .  .  .  .
2  .  .  .  .  .
3  .  X  .  .  X
4  .  .  .  .  .
Au tour du random
   0  1  2  3  4
0  O  O  X  O  .
1  .  .  .  .  .
2  .  .  .  .  .
3  .  X  .  .  X
4  X  .  .  .  .
Au tour de l'IA
   0  1  2  3  4
0  O  O  X  O  O
1  .  .  .  . 